# 🧠 Face Mask Detection in Google Colab (Jupyter Notebook)
This notebook trains a CNN model to detect face masks using Keras and OpenCV.

In [ ]:
# ✅ Install required packages
!pip install tensorflow keras opencv-python pillow pandas

In [ ]:
# ✅ Import Libraries
import numpy as np
import cv2
import datetime
import pandas as pd
import os
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from google.colab import files
from IPython.display import Image, display
from PIL import Image as PILImage

In [ ]:
# ✅ Mount Google Drive (if your dataset is in Drive)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ✅ Define Dataset Paths
train_path = "/content/drive/MyDrive/FaceMaskDataset/train"
test_path = "/content/drive/MyDrive/FaceMaskDataset/test"

In [ ]:
# ✅ Define and Train Model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# ✅ Prepare Data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary')

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary')

In [ ]:
# ✅ Train the Model
model.fit(training_set, epochs=5, validation_data=test_set)
model.save("mymodel.h5")
print("✅ Training complete and model saved.")

In [ ]:
# ✅ Upload image for prediction
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

In [ ]:
# ✅ Predict the uploaded image
test_image = load_img(img_path, target_size=(150, 150))
display(test_image)

test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

mymodel = load_model("mymodel.h5")
prediction = mymodel.predict(test_image)[0][0]

if prediction == 1.0:
    print("❌ MASK NOT FOUND")
else:
    print("✅ MASK FOUND")